# Engaging Complexity: Introduction to Information Theory

<table style="background-color: white;">
<tr>
<td>
<img src="binary_symmetric.png" alt="binary symmetric channel" style="width: 80%;"/>
</td>
<td>
<img src="Shannon.jpeg" alt="Claude Shannon" style="width: 120%;"/>
</td>
</tr>
</table>


In [145]:
import requests
import pandas as pd

import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from nltk import word_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
stops = set(stopwords.words('english'))
import string
punct = list(string.punctuation)
from collections import Counter
import numpy as np
import scipy
from scipy.stats import entropy
import seaborn as sns
import matplotlib.pyplot as plt
from math import comb

[nltk_data] Downloading package punkt to /Users/james/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/james/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/james/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Topic 1: Probability distributions

## 1. What is a probability distribution?

A probability distribution is a representation of how likely it is that a variable can take a particular value, for all possible values that the variable can take. There are two types of probability distribution:

* A discrete probability distribution is defined in a variable that can take a finite number of possible states
* A continuous probability distribution is defined an a variable that can take an infinte number os possible states.

A probability can take a value between 0 and 1 inclusive. A value of 0 means that it is impossible for a variable to take that state; a value of 1 means that taking that state is guaranteed. Because a probability distribution is defined across all possible states of a variable, its values must sum to 1 (i.e. it is guaranteed that the system will be in <i>some</i> state).

### Example 1: A discrete probability distribution

<img src="NUTS.png" alt="UK NUTS regions" style="width: 40%;"/>

Let's imagine we're interested we pick a random resident of the UK. There are only a finite number of regions they can be from. Once we know the population of each region, we can work out the probility that they are resident in that region by dividing by the regional population by the total UK population:

$$P(resident\;in\;region) = \frac{regional\;population}{UK\;population}$$

According to the Office of National Statistics, the UK population is 67,025,542. Given regional populations, this gives the following distribution:


| Region | Population | Probability |
|:---------|:--------:|:--------:|
| Scotland   |  5463992   | 0.081|
| North East  |  2673468   |  0.039|
|  North West  |  7355476   | 0.109|
|  Yorkshire and<br>the Humber  | 5517920  | 0.082 |
|  Northern<br>Ireland | 1898785 | 0.028 |
|  East<br>Midlands | 4861236 | 0.072 |
|  West<br>Midlands | 5962551| 0.088 |
|  East of<br>England | 6259318| 0.093 |
|  London| 9004875| 0.134 |
|  South East| 9212113| 0.137 |
|  South West | 5660791|0.084 |
|  Wales | 3155017|0.047 |

Charting this on a barplot gives us a graphical representation of our discrete probability distribution:

<img src="populations_.png" alt="chart" width="600" height="400"/>


## Example 2: A continuous probability distribution

Let's imagine we're interested in the probability of a random person being a partcular height. While there are physical limitations on how tall a human being can be (the record is 272cm), every human has been every value of height smaller than their current height. This means that height varies continuously. Height also forms a normal distribution, which means that extreme values are uncommon.

<img src="Height.png" alt="height" style="width: 60%;"/>

Though we will be dealing with discrete distributions, are two features of continuous probability distributions you should probably be aware of:

* Where the sum of proabilities in a discrete distribution is 1, the area under the curve in a continuus distributions sums to 1.
* Because a continuous distribution technically contains an infinite number of values, the probability of any one exact value occurring is 0. Instead, probability is defined in a range of values––see below.

<img src="Height_strip.png" alt="height" style="width: 60%;"/>

## 2. Joint probability distributions

A joint probability is defined across the states of two variables. It registers the probability of every possible pair of states of the two variables occurring. Let's explore with an example.

Every resident of the UK lives in a specfic region; we have already worked out the probability of a random person living in each of these regions. However, every UK resident also has an ethnicity. Census data records these as follows:

| Ethnicity | Population | Probability |
|:---------|:--------:|:--------:|
| Asian   |  5026915.6  | 0.075|
| Black  |  2211842.8   |  0.033|
|  Mixed  |  1474561.9   | 0.022|
|  Other  | 670255.4  | 0.01 |
|  White | 57641966.1 | 0.86 |


A joint probability distribution across region of residence ($X$) and ethnicity ($Y$) would list the probility of random person being of a specific ethnicity and living in a specific region, for all values of region and ethnicity. How could be calculate this? 

This is done using the law of $AND$, or the multiplication rule. Namely, for any two independent events, the probability of their joint occurence is the product of their individual probabilities. (The law of $OR$ states that the probability of one OR another independent events occurring is ascertained by adding their probabilities.)

Let's take an example: What's the probability of residing in the North West and being of Asian ethnicity?

$$P(X=North\;West)=0.109$$
$$P(Y=Asian)=0.075$$
$$P(Y=North\;West,\;Y=Asian)=0.109\times{0.075}=0.008175$$



In [90]:
total_pop

67025542

### Let's create a dataframe that gives us the joint probabilities for all pairs of values for region and ethnicity:

In [275]:
UK_regions = {'Scotland': 5463992, 'North East': 2673468, 'North West': 7355476, 'Yorkshire and the Humber': 5517920, \
              'Northern Ireland': 1898785, 'East Midlands': 4861236, 'West Midlands': 5962551, \
              'East of England' : 6259318, 'London': 9004875, 'South East': 9212113, 'South West': 5660791, \
              'Wales': 3155017}
total_pop = sum([i for i in UK_regions.values()])
region_prob = [i/total_pop for i in UK_regions.values()]

UK_ethnicity = {'Asian': 0.075, 'Black': 0.033, 'Mixed': 0.022, 'Other': 0.01, 'White': 0.86}
ethnic_prob = list(UK_ethnicity.values())


table = [[] for x in range(len(UK_regions.keys()))]

for i in range(len(UK_regions.keys())):
    for j in ethnic_prob:
        table[i].append(region_prob[i]*j)
    

joint_distribution = pd.DataFrame(table, columns = UK_ethnicity.keys(), index = UK_regions.keys())





In [ ]:
joint_distribution

## In-class exercise

1. What day of the week were you born on? [Go here to find out](https://www.timeanddate.com/date/weekday.html)
2. Is the day of the month you were born in an odd or even number?

Now, let's create a dataframe with these values for the whole class.

In [166]:
days = {'Monday': 2, 'Tuesday': 5, 'Wednesday': 6, 'Thursday': 3, 'Friday': 3, 'Saturday': 2, 'Sunday': 5}
oddness = {'Odd': 16, 'Even': 10}

total = sum([i for i in days.values()])
days_prob = [i/total for i in days.values()]
oddness_prob = [i/total for i in oddness.values()]

table = [[] for x in range(len(days.keys()))]

for i in range(len(days.keys())):
    for j in oddness_prob:
        table[i].append(days_prob[i]*j)
    

class_distribution = pd.DataFrame(table, columns = oddness.keys(), index = days.keys())


#### Some questions:

1. What is the probability of a person having an odd birthday on a weekend?
2. Select two people at random. What's the probability they both share a non-weekend birthday OR an odd birthday?
3. Select two people at random. What's the probability they both share a the same day of the week as a birthday?


# Topic 2: Shannon entropy

### Surprise

What is entropy? There are several kinds, but the two most common are <b>thermodynamic entropy</b> and <b>information-theoretic entropy</b>. Thermyodynamic entropy is a <b>measure of disorder</b> in a system; information-theoretic entropy (called Shannon entropy) is a <b>measure of unpredicatbility</b> in a system. Though both concepts are related, we are here interested in Shannon entropy. 

The first step in understanding entropy comes with defining the notion of <b>surprise</b> (sometimes called surprisal, information, or self-information). When is an event surprising? When it's unlikely but happens anyway. Therefore, surprise is inveresely proportional to probability: high probability events have low surprise (we expect them to occur) while low probability events have high surprise (we don't expect them to occur).

Let's take an example. What would surprise us most if it fell from the sky––frogs, ash, snow, or rain?

<img src="frogs.png" alt="frogs" style="width: 60%;"/>

The answer, obviously, is frogs––but how can we quantify this? We need a function that, in the $0$ to $1$ range, makes small probabilities large and large probabilities small. Do we have such a function?

Yes we do! The negative of the logarithmic function does exactly this. That is:

$$S = -\log_{2}(p)$$

If we define surprise in this way and plot the results, we can quickly see why it works:

<img src="surprise.png" alt="surprise" style="width: 60%;"/>


### Expected value

The second step in understanding entropy comes with the idea of <b>expected value</b>. This captures the long-term output of a system. It is calculated by mutiplying the probability of system's state by the output of the system's state, and adding the results for all the states. 

Imagine my system, $X$ is a biased coin, with $P(heads) = 0.75$ and $P(tails) = 0.25$. Now, further suppose that I get £25 every time I get a head, and £35 every time I get a tail. My exepcted value for $X$, $E[X]$ is given by:

$$E[X] = (0.75\times{25})+(0.25\times{35})= £27.5$$

That is, the long-run average of my takings will trend towards £27.5. We can see this by simulating 1,000 trials:

<img src="expected_value_.png" alt="ev" style="width: 60%;"/>

### Definition: Entropy is the expected value of surprise

We can now attempt to define Shannon entropy, often denoted by $H$. Entropy is a measure of unpredictability and therefore of surprise; we have already seen how $-log_{2}(p)$ quantifies surprise for a single event. In entropy, we are interested in the surprise associated with all the states (i.e. events) of a specfic system. In other words, <b>entropy is the expected value of surprise</b>. To calculate it, we multiply the surprise of a state by its probability of occurring, and add up the results for every state of the system. 

This is the formula for entropy, where $H$ is the entropy measure and $X$ is a discrete probability distribution over $n$ states of a system:

$$X = (x_1, x_2, x_3, ... x_n)$$

$$H(X) = -\sum_{x \in X} p(x)\log_{2} p(x)$$

The $\sum_{x \in X}$ notation here simply means "add up the results for every value of $x$ in $X$". 

## Example

<img src="urn.png" alt="ev" style="width: 30%;"/>


What does this look like in practice? Let's take an example. An urn contains 7 red balls and 3 blue balls. What is the entropy of this system $X$? The answer comes with recognising that it defines a probability distribution across colours:

$P(X=red):\frac{7}{10}$

$P(X=blue):\frac{3}{10}$

$$H(X) = -\begin{bmatrix}{\frac{7}{10}\times \log_{2}(\frac{7}{10}) + \frac{3}{10}\times \log_{2}(\frac{3}{10})}\end{bmatrix}$$

$$H(X) = -[-0.5210896782498619 - 0.3602012209808308]$$

$$H(X) = 0.8812908992306927\;bits$$

Compare with an urn that contains 5 red balls and 5 blue balls:

$$H(X) = -\begin{bmatrix}{\frac{5}{10}\times \log_{2}(\frac{5}{10}) + \frac{5}{10}\times \log_{2}(\frac{5}{10})}\end{bmatrix}$$

$$H(X) = -[-0.5 - 0.5]$$

$$H(X) = 1\;bits$$

Because the urn with equal numbers of red and blue balls is harder to predict, it has higher entropy than the system with more red than blue balls. 

### Calculating entropy using `scipy`

Usefully, python allows us to calculate entropy easily using the `scipy` library. [Find the docs here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.entropy.html).

In [171]:
from scipy.stats import entropy

regions_e = entropy(region_prob)
ethnic_e = entropy(ethnic_prob)

print("Entropy of probability distribution of residing in UK region in bits:", regions_e)
print("Entropy of probability distribution of belonging to ethnicity in bits:", ethnic_e)

Entropy of probability distribution of residing in UK region in bits: 2.4038891997141607
Entropy of probability distribution of belonging to ethnicity in bits: 0.5665682812818006


# Topic 3: Problem-solving using entropy

### 1. Distinguishing between language and noise

One particular value of entropy is in distinguishing between in information containing signal and noise. For a signal to be present, it must be encoded––and encoding implies repetition. Here, we will explore an example of how we might use entropy to do this for written text.

1. We we will split two texts––Emily Brontë's <i>Wuthering Heights</i> and William Shakespeare's <i>Collected Works</i>––into their constituent words.
2. We will generate 25 texts made up of random strings of characters of characters of random length.
3. We will cryptographically hash each word of each text using the SHA-256 algorithm to make them indistinguishable to the human eye.
4. We will scramle the list of texts so we don't know where the real and the fake ones are.
5. We will measure the entropy of the resulting hashed word frequency distributions to see if we can identify the real texts.

In [ ]:
#Import and demo the hashing library

import hashlib

input_string = "Hello, world!"

# Create a SHA-256 hash object
hash_object = hashlib.sha256()
hash_object.update(input_string.encode())

hex_dig = hash_object.hexdigest()

print(hex_dig)


In [265]:
#Open and extract words from our literary texts

with open('wuthering_heights.txt', 'r') as f:
    text = f.read()
    
text = text.encode('ascii', 'ignore')
text = text.decode()
text = ' '.join(text.splitlines())
text = text.lower()
words = word_tokenize(text)
lemmas_wh = [lemmatizer.lemmatize(i) for i in words]

with open('shakespeare_cw.txt', 'r') as f:
    text = f.read()
    
text = text.encode('ascii', 'ignore')
text = text.decode()
text = ' '.join(text.splitlines())
text = text.lower()
words = word_tokenize(text)
lemmas_sh = [lemmatizer.lemmatize(i) for i in words]
lemmas_sh = lemmas_sh[:len(lemmas_wh)]

In [273]:
#Generate our noisy texts

import random
import string

def randoms(n, min_length, max_length):
    random_strings = []
    for _ in range(n):
        length = random.randint(min_length, max_length)
        random_str = ''.join(random.choice(string.ascii_lowercase) for _ in range(length))
        random_strings.append(random_str)
    return random_strings

n = len(lemmas_wh)  # Number of strings
min_length = 1  # Minimum length of strings
max_length = 12  # Maximum length of strings

texts = []

for i in range(25):
    texts.append(randoms(n, min_length, max_length))







In [268]:
#Shuffle and then ryptographically hash all our texts

import random
all_text = texts + [lemmas_wh] + [lemmas_sh]
random.shuffle(all_text)

hashes = [[] for i in range(len(all_text))]

for i in range(len(all_text)):
    for j in all_text[i]:
        hash_object = hashlib.sha256()
        hash_object.update(j.encode())
        hex_dig = hash_object.hexdigest()
        hashes[i].append(hex_dig)
    



In [269]:
#Get counts of hashed words and create probability distribution

from collections import Counter

distributions = []

for i in hashes:
    item_counts = Counter(i)
    prob = [j/len(i) for j in item_counts.values()]
    distributions.append(prob)





In [ ]:
#Calculate and plot entropies for each text

entropies = [entropy(i, base = 2) for i in distributions]
entropies = pd.Series(entropies)

sns.barplot(x = entropies.index, y = entropies.values)
plt.xlabel('hashed text ID')
plt.ylabel('bits')